<a href="https://colab.research.google.com/github/aubricot/object_detection_for_image_cropping/blob/master/train_tf_ssd_rcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Tensorflow Object Detection models
---   
*Last Updated 29 January 2020*  
Use images and bounding box coordinates to train Faster-RCNN and SSD Object Detection Models implemented in Tensorflow to detect bats from EOL images.

Datasets exported from [preprocessing.ipynb](https://colab.research.google.com/github/aubricot/object_detection_for_image_cropping/blob/master/preprocessing.ipynb) were already downloaded to Google Drive in preprocessing.ipynb. 

For each 24 hour period on Google Colab, you have up to 12 hours of GPU access. Training the object detection model on bats took 30 hours split into 3 days.

Make sure to set the runtime to Python 2 with GPU Hardware Accelerator.    

References:   
[Official Tensorflow Object Detection API Instructions](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html)   
[Medium Blog on training using Tensorflow Object Detection API in Colab](https://medium.com/analytics-vidhya/training-an-object-detection-model-with-tensorflow-api-using-google-colab-4f9a688d5e8b)

## Installs (run this every time)
---
Install the Tensorflow Object Detection API directly to this Colab notebook.

In [0]:
# Mount google drive to import/export files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
# Download, compile and build all ingredients for the Tensorflow Object Detection API
# These steps take a couple of minutes and print a lot of output

# Make a working directory train/ in your Google Drive and include the path here (all paths in other sections stay the same)
import os
%cd drive/My Drive/fall19_smithsonian_informatics/train
if not os.path.exists("tf_models"):
  !mkdir tf_models
%cd tf_models

# Install Tensorflow Object Detection API
import pathlib
if not pathlib.Path("models").exists():
  !git clone https://github.com/tensorflow/models.git

# Clone the COCO API repository to your Google Drive
if not pathlib.Path("pycocotools").exists():
  !git clone https://github.com/cocodataset/cocoapi.git
  # Move needed folders to tf_models/pycocotools and delete remaining contents of cocoapi/ to save space
  !cd cocoapi/PythonAPI; make; cp -r pycocotools ../..
  !rm -rf cocoapi

# Install libraries
!apt-get install -qq protobuf-compiler python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive

# Compile object detection api using Google Protobuf
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.

# Update system path variables
os.environ['PYTHONPATH'] = ':/drive/My Drive/fall19_smithsonian_informatics/train/tf_models/models/research/:/drive/My Drive/fall19_smithsonian_informatics/train/tf_models/models/research/slim/'
!echo $PYTHONPATH

import sys
print(sys.path)
sys.path.append("/usr/local/lib/python2.7/dist-packages/tensorflow/contrib/slim")
print(sys.path)

# Build slim
!python slim/setup.py build
!python slim/setup.py install

# Copy slim to specified directories to avoid errors in model_builder_test.py
!cp -R models/research/slim/ /usr/local/lib/python2.7/dist-packages/object_detection-0.1-py2.7.egg/
if not os.path.exists("object_detection/slim/nets"):
  !cp -R slim/nets/ object_detection/

# Test build of model
!python object_detection/builders/model_builder_test.py

## Other preparation for training (only need to run these once)
---

### Download and extract pre-trained model   

In [0]:
# Download pre-trained model from Tensorflow Object Detection Model Zoo
# SSD and Faster-RCNN both included as options below
# modified from https://github.com/RomRoc/objdet_train_tensorflow_colab/blob/master/objdet_custom_tf_colab.ipynb

# cd to train/
#%cd train
%cd ../../..

import os
import shutil
import glob
import urllib
import tarfile

# Make folders for your training files for each model
# RCNN Model
if not (os.path.exists('tf_models/train_demo')):
  !mkdir tf_models/train_demo
if not (os.path.exists('tf_models/train_demo/rcnn')):
  !mkdir tf_models/train_demo/rcnn
if not (os.path.exists('tf_models/train_demo/rcnn/pretrained_model')):
  !mkdir tf_models/train_demo/rcnn/pretrained_model
if not (os.path.exists('tf_models/train_demo/rcnn/finetuned_model')):
  !mkdir tf_models/train_demo/rcnn/finetuned_model
if not (os.path.exists('tf_models/train_demo/rcnn/trained')):
  !mkdir tf_models/train_demo/rcnn/trained
# Download the model
MODEL = 'faster_rcnn_resnet50_coco_2018_01_28'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = 'tf_models/train_demo/rcnn/pretrained_model'
if not (os.path.exists(MODEL_FILE)):
  opener = urllib.URLopener()
  opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
  shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

# SSD Model
if not (os.path.exists('tf_models/train_demo/ssd')):
  !mkdir tf_models/train_demo/ssd
if not (os.path.exists('tf_models/train_demo/ssd/pretrained_model')):
  !mkdir tf_models/train_demo/ssd/pretrained_model
if not (os.path.exists('tf_models/train_demo/ssd/finetuned_model')):
  !mkdir tf_models/train_demo/ssd/finetuned_model
if not (os.path.exists('tf_models/train_demo/ssd/trained')):
  !mkdir tf_models/train_demo/ssd/trained
# Download the model
MODEL = 'ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = 'tf_models/train_demo/ssd/pretrained_model'
if not (os.path.exists(MODEL_FILE)):
  opener = urllib.URLopener()
  opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
  shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

### Convert training data to tf.record format

1) Download generate_tfrecord.py from [GitHub](https://github.com/aubricot/object_detection_for_image_cropping/blob/master/generate_tfrecord.py)

2) Modify the file for your train dataset: 
*   update label names to your class(es) at line 34
        # TO-DO replace this with label map
        def class_text_to_int(row_label):
          if row_label == 'Chiroptera':
            return 1
          else:
            None
*   update the filepath where you want your train tf.record file to save at line 88
        # TO-DO replace path with your filepath
        def main(_):
            writer = tf.python_io.TFRecordWriter('/content/drive/My Drive/[yourfilepath]/tf.record')
*   upload modified generate_tfrecord.py file to train/

3) Proceed with steps below to generate tf.record files for your test and train datasets

In [0]:
# Convert crops_train to tf.record format for train data
# Modified from https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html
!python generate_tfrecord.py --csv_input='/content/drive/My Drive/fall19_smithsonian_informatics/train/chiroptera_crops_train.csv'  --output_path= "/content/drive/My Drive/fall19_smithsonian_informatics/train/images/tf.record"  --image_dir="/content/drive/My Drive/fall19_smithsonian_informatics/train/images"

In [0]:
# Convert crops_test to tf.record format for test data
# Modified from https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html
!python generate_tfrecord.py --csv_input='/content/drive/My Drive/fall19_smithsonian_informatics/train/chiroptera_crops_test.csv'  --output_path= "/content/drive/My Drive/fall19_smithsonian_informatics/train/test_img/tf.record"  --image_dir="/content/drive/My Drive/fall19_smithsonian_informatics/train/test_img"

## Train
--- 

In [0]:
# Track training checkpoints in external window
# Modified from https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/
LOG_DIR = 'training'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
# Install
! npm install -g localtunnel
! npm i -g npm
# Tunnel port 6006 (TensorBoard assumed running)
get_ipython().system_raw('lt --port 6006 >> url1.txt 2>&1 &')
# Get url
! cat url1.txt

In [0]:
# Make label_map.pbtxt file for one class
## this doesnt work...need to figure out way to make this like labelmap.pbtxt
!echo "item {\n id: 1\n name: 'Chiroptera'\n}" > label_map.pbtxt

In [0]:
# cd to train/
#%cd train
%cd ../../..

In [0]:
# Modified from https://github.com/RomRoc/objdet_train_tensorflow_colab/blob/master/objdet_custom_tf_colab.ipynb
# To train on rcnn or ssd, change the pipline_config_path to the appropriate config file
import matplotlib
matplotlib.use('Agg')

!python tf_models/models/research/object_detection/model_main.py \
    --pipeline_config_path=tf_models/models/research/object_detection/samples/configs/ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03_bats.config \
    --alsologtostderr \
    --num_train_steps=190000 \
    --num_eval_steps=500 \
    --model_dir=tf_models/train_demo/ssd/trained/ \

In [0]:
# Export trained model
# Modified from https://github.com/RomRoc/objdet_train_tensorflow_colab/blob/master/objdet_custom_tf_colab.ipynb
# Adjust pipeline_config_path and output_directory when switching between ssd and faster rcnn
%cd train

lst = os.listdir('tf_models/train_demo/ssd/trained')
lf = filter(lambda k: 'model.ckpt-' in k, lst)
last_model = sorted(lf)[-1].replace('.meta', '')

!python tf_models/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=tf_models/models/research/object_detection/samples/configs/ssd/ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03_bats.config \
    --output_directory=tf_models/train_demo/rcnn/finetuned_model \
    --trained_checkpoint_prefix=tf_models/train_demo/ssd/trained/$last_model

### Test trained model on test images
--- 

In [0]:
%tensorflow_version 1.0

import tensorflow as tf 
tf.compat.v1.enable_eager_execution()

# For importing/exporting files, working with arrays, etc
import os
import pathlib
import six.moves.urllib as urllib
import sys
import tarfile
import zipfile
import numpy as np 
import csv
import matplotlib
import time
import pandas as pd

# For downloading the images
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO
from collections import defaultdict
from io import StringIO

# For drawing onto and plotting the images
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

import cv2

from IPython.display import display

sys.path.append("tf_models/models/research/")
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
%matplotlib inline

In [0]:
# Load in trained model files
%cd train

# Path to frozen detection graph. This is the actual model that is used for the object detection.
# SSD Model
PATH_TO_CKPT = 'tf_models/train_demo/ssd/finetuned_model' + '/frozen_inference_graph.pb'
# Faster RCNN Model
#PATH_TO_CKPT = 'tf_models/train_demo/rcnn/finetuned_model' + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'labelmap.pbtxt'

NUM_CLASSES = 1
    
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.compat.v1.GraphDef()
  with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images/'
names = os.listdir(PATH_TO_TEST_IMAGES_DIR)
#TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 2) ]
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, name) for name in names]
print(TEST_IMAGE_PATHS)


## Define function for object detection
---   
Use a Tensorflow session to detect objects using pre-trained models and display the results of detection.

In [0]:
def show_inference(image_np_expanded):
  with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
      # Definite input and output Tensors for detection_graph
      image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
      # Each box represents a part of the image where a particular object was detected.
      detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
      #max_boxes_to_draw = detection_boxes.shape[0] # add this line and remove (i) and (ii) below to show multiple detection boxes
      # Each score represent how level of confidence for each of the objects.
      # Score is shown on the result image, together with the class label.
      detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
      detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
      num_detections = detection_graph.get_tensor_by_name('num_detections:0')
      min_score_thresh = .7

      # Actual detection.
      (boxes, scores, classes, num) = sess.run(
          [detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      
      # Visualization of the results of a detection.
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          min_score_thresh=.7,
          max_boxes_to_draw=1,
          line_thickness=8)
      
      plt.figure()
      plt.imshow(image_np)

## Load in test images and run them through the trained object detector
---

In [0]:
# Loops through first 5 image urls from the text file
for image_path in TEST_IMAGE_PATHS[:20]:
 
    # Record inference time
    image = Image.open(image_path)
    image_np = load_image_into_numpy_array(image)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    start_time = time.time()
    show_inference(image_np_expanded)
    end_time = time.time()
    #print('Detection complete in {} of 145 test images'.format(i+1))

    #add way to plot mAP
    #plt.title('{}) Inference time: {}'.format(i+1, format(end_time-start_time, '.2f'))) 